In [34]:
import sys
import random
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
import sumolib
import traci
from sumolib import checkBinary
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import sys
import io
from contextlib import redirect_stdout
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
from utils import *


if 'SUMO_HOME' in os.environ:
    print('SUMO_HOME found')
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))

sumoBinary = checkBinary('sumo-gui')
# sumoBinary = checkBinary('sumo')
roadNetwork = "./config/osm.sumocfg"
sumoCmd = [sumoBinary, "-c", roadNetwork, "--start", "--quit-on-end"]
# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

SUMO_HOME found
Using device: cuda


In [38]:
planned_path = get_planned_path()
path_values = list(planned_path.values())
path_values = torch.tensor(path_values).float() / 10

TraCIException: Connection 'default' is already active.

In [36]:
def project_to_nearest(prediction, planned_path):
    with torch.no_grad():
        starts = planned_path[:, :-1, :]
        to = planned_path[:, 1:, :]

        prediction = prediction.unsqueeze(1).repeat(1, starts.shape[1], 1)
        ap = prediction - starts
        ab = to - starts
        numerator = torch.einsum('ijk,ijk->ij', ap, ab)
        denominator = torch.einsum('ijk,ijk->ij', ab, ab)
        t = numerator / denominator
        t = torch.nan_to_num(t, nan=0.0)
        t = torch.clamp(t, 0, 1)
        projections = starts + t.unsqueeze(2) * ab
        diff = projections - prediction
        distances = torch.norm(diff, dim=2)
        min_indices = torch.argmin(distances, dim=1)
        projections = projections[range(projections.shape[0]), min_indices]
        return projections, min_indices


def project_to_nearest_with_checkpoints(prediction, planned_path):
    with torch.no_grad():
        starts = planned_path[:, :-1, :]
        to = planned_path[:, 1:, :]

        prediction = prediction.unsqueeze(1).repeat(1, starts.shape[1], 1)
        ap = prediction - starts
        ab = to - starts
        numerator = torch.einsum('ijk,ijk->ij', ap, ab)
        denominator = torch.einsum('ijk,ijk->ij', ab, ab)
        t = numerator / denominator
        t = torch.nan_to_num(t, nan=0.0)
        t = torch.clamp(t, 0, 1)
        projections = starts + t.unsqueeze(2) * ab
        diff = projections - prediction
        distances = torch.norm(diff, dim=2)
        min_indices = torch.argmin(distances, dim=1)
        projections = projections[range(projections.shape[0]), min_indices]
        next_checkpoint = to[range(to.shape[0]), min_indices]
        pad_to = F.pad(to, (0, 0, 1, 0), value=0)
        next_next_checkpoint = pad_to[range(to.shape[0]), min_indices + 1]
        projections_with_checkpoints = torch.cat((projections, next_checkpoint, next_next_checkpoint), dim=1)
        return projections_with_checkpoints

In [37]:
# Example DataFrame loading
df = pd.read_csv('edinburgh_trajectories.csv')
num_cols = df.shape[1]
position_indices = [i for i in range(num_cols) if i % 4 == 1 or i % 4 == 2]
position_df = df.iloc[:, position_indices]
position_array = position_df.to_numpy()
sequence_length = len(position_indices) // 2
tensor_list = []

for row in position_array:
    reshaped_tensor = torch.tensor(row.reshape(sequence_length, 2))
    tensor_list.append(reshaped_tensor)

all_trajectories_tensor = torch.stack(tensor_list).float() / 10

next_checkpoint = torch.zeros_like(all_trajectories_tensor)
next_next_checkpoint = torch.zeros_like(all_trajectories_tensor)
path_values_pad_1 = F.pad(path_values, (0, 0, 1, 0))
for i in range(all_trajectories_tensor.shape[1]):
    _, min_indices = project_to_nearest(all_trajectories_tensor[:, i], path_values_pad_1)
    next_checkpoint[:, i] = path_values_pad_1[range(path_values_pad_1.shape[0]), min_indices+1]
    next_next_checkpoint[:, i] = path_values_pad_1[range(path_values_pad_1.shape[0]), min_indices+2]

all_trajectories_tensor = torch.cat((all_trajectories_tensor, next_checkpoint, next_next_checkpoint), dim=2)

def missing(x, y, z, p):
    return torch.tensor(np.repeat(np.random.rand(x * y) < p, z).reshape(x, y, z)).float()

def generate_masks(tensors, min_mask_ratio=0.2, max_mask_ratio=0.4, missing_ratio=0.5, complete_traj_ratio=0.8):
    initial_masks = missing(tensors.shape[0], tensors.shape[1], tensors.shape[2], missing_ratio)
    masks = []
    for initial_mask in initial_masks:
        if np.random.rand() < complete_traj_ratio:
            masks.append(torch.zeros_like(initial_mask).tolist())
            continue
        seq_length = initial_mask.shape[0]
        mask_start = np.random.randint(int(seq_length * min_mask_ratio), int(seq_length * max_mask_ratio))
        mask = torch.zeros_like(initial_mask)
        mask[:, :mask_start] = 1
        mask = initial_mask * mask
        mask[0] = 0
        mask[1] = 0
        masks.append(mask.tolist())
    return torch.tensor(masks)

# split the data into training and validation sets
# because the data is randomly generated, we don't need to shuffle it
train_ratio = 0.8
train_size = int(train_ratio * all_trajectories_tensor.shape[0])
train_trajectories_tensor = all_trajectories_tensor[:train_size]
val_trajectories_tensor = all_trajectories_tensor[train_size:]

train_path_values = path_values[:train_size]
val_path_values = path_values[train_size:]

train_mask = generate_masks(train_trajectories_tensor)

class DatasetWithPlans(Dataset):
    def __init__(self, tensor, input_mask, path_values):
        self.tensor = tensor.float().to(device)
        self.input_mask = input_mask.float().to(device)
        self.path_values = path_values.float().to(device)
    def __len__(self):
        return len(self.tensor)
    def __getitem__(self, idx):
        return self.tensor[idx], self.input_mask[idx], self.path_values[idx]

train_dataset = DatasetWithPlans(train_trajectories_tensor, train_mask, train_path_values)

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, int(hidden_size/2))
        self.fc2 = nn.Linear(int(hidden_size/2), num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x, h0=None):
        if h0 is None:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, hidden = self.gru(x, h0)  
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        return out, hidden

class CustomMSE(nn.Module):
    def __init__(self):
        super(CustomMSE, self).__init__()

    def forward(self, output, target):
        # where target is 0
        mask = (target != 0).float().to(device)
        mse = torch.mean(((output - target) ** 2) * mask)
        directional_diff = torch.mean(torch.abs(torch.atan2(output[:, 1], output[:, 0]) - torch.atan2(target[:, 1], target[:, 0])))
        return mse + directional_diff

In [ ]:
def train_model(model, dataloader, epochs, optimizer, criterion):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, masks, planned_path in dataloader:
            optimizer.zero_grad()
            hidden = None
            loss = 0
            # Autoregressive prediction
            # Start with the first input and predict each subsequent step
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1)
            for t in range(1, seq_len):
                prediction, hidden = model(current_input, hidden)

                previous_input = inputs[:, t-1, :2]
                # if epoch > 30:
                projection_with_checkpoints = project_to_nearest_with_checkpoints(prediction, planned_path)
                current_input = (projection_with_checkpoints * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
                # else:
                #     current_input = inputs[:, t, :].unsqueeze(1)
                loss += criterion(prediction - previous_input, inputs[:, t, :2]-previous_input) #(current_input-previous_input).squeeze(1))
                
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
 
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

model = GRUModel(input_size=6, hidden_size=128, num_layers=2, num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomMSE()
train_model(model, train_dataloader, 50, optimizer, criterion)

Epoch 1, Loss: 1389392.51875
Epoch 2, Loss: 1193120.3375
Epoch 3, Loss: 856382.703125
Epoch 4, Loss: 548533.384375
Epoch 5, Loss: 369493.1984375
Epoch 6, Loss: 301424.8890625
Epoch 7, Loss: 290142.91328125
Epoch 8, Loss: 220131.61328125
Epoch 9, Loss: 136830.056640625
Epoch 10, Loss: 94724.487109375
Epoch 11, Loss: 72619.285546875
Epoch 12, Loss: 62818.7234375
Epoch 13, Loss: 52136.91796875
Epoch 14, Loss: 36351.49580078125
Epoch 15, Loss: 25078.015966796876
Epoch 16, Loss: 18890.2984375
Epoch 17, Loss: 14341.253588867188
Epoch 18, Loss: 12198.678564453125
Epoch 19, Loss: 10424.952197265626
Epoch 20, Loss: 9554.779931640625
Epoch 21, Loss: 8682.198828125
Epoch 22, Loss: 8483.322094726562
Epoch 23, Loss: 8226.910546875
Epoch 24, Loss: 7913.778955078125
Epoch 25, Loss: 7850.6739501953125
Epoch 26, Loss: 7613.5089111328125
Epoch 27, Loss: 7702.5389404296875
Epoch 28, Loss: 7551.488720703125
Epoch 29, Loss: 7420.055444335938
Epoch 30, Loss: 7553.964599609375
Epoch 31, Loss: 7585.2848632812

In [ ]:
def checkModel(model, inputs, masks, paths):
    model.eval()
    with torch.no_grad():
        hidden = None
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            prediction, hidden = model(current_input, hidden)
            projection_with_checkpoints = project_to_nearest_with_checkpoints(prediction, paths)
            current_input = (projection_with_checkpoints * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            print(projection_with_checkpoints[2][:2], inputs[:, t, :][2][:2])

# get the first batch in dataloader
val_mask = generate_masks(val_trajectories_tensor, missing_ratio=0.9, complete_traj_ratio=0)
val_dataset = DatasetWithPlans(val_trajectories_tensor, val_mask, val_path_values)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
inputs, masks, paths = next(iter(val_dataloader))

checkModel(model, inputs, masks, paths)

tensor([49.7986,  6.9225], device='cuda:0') tensor([48.6204,  7.5482], device='cuda:0')
tensor([48.2167,  8.5929], device='cuda:0') tensor([48.3590,  7.8862], device='cuda:0')
tensor([52.7940, 10.5730], device='cuda:0') tensor([47.9903,  8.3632], device='cuda:0')
tensor([54.1586, 11.9705], device='cuda:0') tensor([47.5392,  8.9467], device='cuda:0')
tensor([55.3092, 13.0401], device='cuda:0') tensor([46.9746,  9.6770], device='cuda:0')
tensor([56.4306, 14.0108], device='cuda:0') tensor([46.4005, 10.4195], device='cuda:0')
tensor([57.8302, 15.0346], device='cuda:0') tensor([45.8109, 11.1821], device='cuda:0')
tensor([59.3202, 16.0427], device='cuda:0') tensor([45.3971, 11.7174], device='cuda:0')
tensor([48.4054, 14.1607], device='cuda:0') tensor([45.1648, 12.0178], device='cuda:0')
tensor([48.2457, 14.1140], device='cuda:0') tensor([45.2760, 12.2756], device='cuda:0')
tensor([48.1849, 14.0962], device='cuda:0') tensor([45.4987, 12.1092], device='cuda:0')
tensor([48.1250, 14.0787], devic

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0
    steps = 0
    for inputs, masks, paths in dataloader:
        hidden = None
        loss = 0
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            new_steps = int(sum((inputs[:, t, :].reshape(-1) != 0).float().to(device))) / 2
            if new_steps == 0:
                break
            steps += new_steps
            prediction, hidden = model(current_input, hidden)
            projection_with_checkpoints = project_to_nearest_with_checkpoints(prediction, paths)
            current_input = (projection_with_checkpoints * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            loss += torch.norm((projection_with_checkpoints[:, :2] - inputs[:, t, :2]) * 10, dim=1) * (inputs[:, t, :2] != 0)[:, 0]
            print(projection_with_checkpoints[2][:2], inputs[:, t, :][2][:2])
        total_loss += loss.sum().item()
    print(f'Validation Loss: {total_loss / steps}')

for i in [0.9, 0.8, 0.7, 0.6, 0.5, 0]:
    val_mask = generate_masks(val_trajectories_tensor, missing_ratio=i, complete_traj_ratio=0)
    val_dataset = DatasetWithPlans(val_trajectories_tensor, val_mask, val_path_values)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
    evaluate_model(model, val_dataloader)

tensor([63.0280, 84.2500], device='cuda:0') tensor([62.7772, 85.9434], device='cuda:0')
tensor([62.8470, 85.7750], device='cuda:0') tensor([62.9322, 85.5679], device='cuda:0')
tensor([63.0950, 85.1720], device='cuda:0') tensor([63.1236, 85.0665], device='cuda:0')
tensor([63.1452, 84.9866], device='cuda:0') tensor([63.0606, 84.3246], device='cuda:0')
tensor([63.1030, 85.1427], device='cuda:0') tensor([62.3807, 83.8327], device='cuda:0')
tensor([63.1570, 84.5450], device='cuda:0') tensor([61.5440, 83.4843], device='cuda:0')
tensor([63.1587, 84.9371], device='cuda:0') tensor([60.7207, 83.1414], device='cuda:0')
tensor([63.0950, 85.1720], device='cuda:0') tensor([59.8706, 82.7874], device='cuda:0')
tensor([63.0950, 85.1720], device='cuda:0') tensor([59.0213, 82.4355], device='cuda:0')
tensor([63.0950, 85.1720], device='cuda:0') tensor([58.1518, 82.0768], device='cuda:0')
tensor([63.0950, 85.1720], device='cuda:0') tensor([57.3312, 81.7381], device='cuda:0')
tensor([63.0950, 85.1720], devic

KeyboardInterrupt: 